In [ ]:
from os import path

# Third-party
from astropy.io import fits
from astropy.table import Table, join
import astropy.coordinates as coord
from astropy.stats import mad_std
from astropy.time import Time
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from thejoker.data import RVData
from thejoker.sampler import JokerParams, TheJoker
from thejoker.plot import plot_rv_curves

from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, RedClump,
                        StarResult, Status, JokerRun, initialize_db)
from twoface.data import APOGEERVData
from twoface.io import load_samples
from twoface.plot import plot_data_orbits

from schwimmbad import MultiPool

In [ ]:
TWOFACE_CACHE_PATH = path.abspath('../cache/')
Session, _ = db_connect(path.join(TWOFACE_CACHE_PATH, 'apogee.sqlite'))
session = Session()

In [ ]:
apogee_id = '2M00572766+8415569'
prior_file = 'P8-32768_prior_samples.hdf5'

In [ ]:
star = session.query(AllStar).join(StarResult, JokerRun, Status)\
                             .filter(AllStar.apogee_id == apogee_id)\
                             .filter(Status.id > 0).limit(1).one()
star

In [ ]:
data = star.apogeervdata()
data = data[np.abs(data.stddev) < 100*u.km/u.s]

In [ ]:
data.stddev

Create _TheJoker_:

In [ ]:
params = JokerParams(P_min=8*u.day, P_max=32768*u.day)

In [ ]:
with MultiPool() as pool:
    joker = TheJoker(params, pool=pool)
    samples = joker.iterative_rejection_sample(data, n_requested_samples=128, 
                                               prior_cache_file=path.join(TWOFACE_CACHE_PATH, prior_file))

In [ ]:
fig = plot_data_orbits(data, samples)